In [ ]:
!unzip /content/blur_images.zip -d /content/blur_images

In [ ]:
import imutils
from imutils import paths

import argparse
import cv2
import os
from google.colab.patches import cv2_imshow
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
data = []
general_path = '/content/blur_images/'

for filename in os.listdir(general_path):
    if filename.endswith("jpg"): 
        # Your code comes here such as 
        data.append(general_path + filename)

print(data)

# With OpenCV Variance of the Laplacian

https://www.pyimagesearch.com/2015/09/07/blur-detection-with-opencv/


In [ ]:
def variance_of_laplacian(image):
	# compute the Laplacian of the image and then return the focus
	# measure, which is simply the variance of the Laplacian
	return cv2.Laplacian(image, cv2.CV_64F).var()


In [ ]:
!mkdir laplacian_processed_images

In [ ]:
THRESHOLD = 100
# loop over the input images
for imagePath in data:
  # load the image, convert it to grayscale, and compute the
  # focus measure of the image using the Variance of Laplacian
  # method
  image = cv2.imread(imagePath)
  #image = imutils.resize(image, width=500)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  fm = variance_of_laplacian(gray)
  text = "Not Blurry"
  color = (0,255,0)
  # if the focus measure is less than the supplied threshold,
  # then the image should be considered "blurry"
  if fm < THRESHOLD:
    text = "Blurry"
    color = (0,0,255)
  # show the image
  cv2.putText(image, "{}: {:.2f}".format(text, fm), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 3)
  cv2_imshow(image)
  key = cv2.waitKey(0)

  name = os.path.split(imagePath)[1]
  #print(name)
  
  #write the processed image to specific folder
  cv2.imwrite(('/content/laplacian_processed_images/'+ name), image)

In [ ]:
#zip folder
!zip -r /content/laplacian_processed_images.zip /content/laplacian_processed_images

In [ ]:
#download zip folder
files.download('/content/laplacian_processed_images.zip')

# With OpenCV and Numpy FFT algorithm

https://www.pyimagesearch.com/2020/06/15/opencv-fast-fourier-transform-fft-for-blur-detection-in-images-and-video-streams/



In [ ]:
def detect_blur_fft(image, size=60, thresh=10, vis=False):
	# grab the dimensions of the image and use the dimensions to
	# derive the center (x, y)-coordinates
	(h, w) = image.shape
	(cX, cY) = (int(w / 2.0), int(h / 2.0))
  
  # compute the FFT to find the frequency transform, then shift
	# the zero frequency component (i.e., DC component located at
	# the top-left corner) to the center where it will be more
	# easy to analyze
	fft = np.fft.fft2(image)
	fftShift = np.fft.fftshift(fft)
 
 # check to see if we are visualizing our output
	if vis:
		# compute the magnitude spectrum of the transform
		magnitude = 20 * np.log(np.abs(fftShift))
		# display the original input image
		(fig, ax) = plt.subplots(1, 2, )
		ax[0].imshow(image, cmap="gray")
		ax[0].set_title("Input")
		ax[0].set_xticks([])
		ax[0].set_yticks([])
		# display the magnitude image
		ax[1].imshow(magnitude, cmap="gray")
		ax[1].set_title("Magnitude Spectrum")
		ax[1].set_xticks([])
		ax[1].set_yticks([])
		# show our plots
		plt.show()
  
  # zero-out the center of the FFT shift (i.e., remove low
	# frequencies), apply the inverse shift such that the DC
	# component once again becomes the top-left, and then apply
	# the inverse FFT
	fftShift[cY - size:cY + size, cX - size:cX + size] = 0
	fftShift = np.fft.ifftshift(fftShift)
	recon = np.fft.ifft2(fftShift)
 
  # compute the magnitude spectrum of the reconstructed image,
	# then compute the mean of the magnitude values
	magnitude = 20 * np.log(np.abs(recon))
	mean = np.mean(magnitude)
	# the image will be considered "blurry" if the mean value of the
	# magnitudes is less than the threshold value
	return (mean, mean <= thresh)

In [ ]:
!mkdir fft_processed_images

In [ ]:
# loop over the input images
for imagePath in data:
  print(imagePath)
  # load the input image from disk, resize it, and convert it to
  # grayscale
  orig = cv2.imread(imagePath)
  orig = imutils.resize(orig, width=500)
  gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
  # apply our blur detector using the FFT
  (mean, blurry) = detect_blur_fft(gray, size=60,
	  thresh=20, vis=True)
  # draw on the image, indicating whether or not it is blurry
  image = np.dstack([gray] * 3)
  color = (0, 0, 255) if blurry else (0, 255, 0)
  text = "Blurry ({:.4f})" if blurry else "Not Blurry ({:.4f})"
  text = text.format(mean)
  cv2.putText(orig, text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
	  color, 2)
  print("[INFO] {}".format(text))
  # show the output image
  cv2_imshow(orig)
  cv2.waitKey(0)

  name = os.path.split(imagePath)[1]
  #print(name)
  
  #write the processed image to specific folder
  cv2.imwrite(('/content/fft_processed_images/'+ name), orig)

  print()
  print()

In [ ]:
#zip folder
!zip -r /content/processed_images.zip /content/fft_processed_images

In [ ]:
#download zip folder
files.download('/content/processed_images.zip')